## IBM Applied Data Science Capstone Course by Coursera

### Opening a New Restaurant in Baku, Azerbaijan

* Build a dataframe of neighborhoods in Baku by reading html  data from website
* Get the geographical coordinates of the neighborhoods
* Show the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new restaurant

###  1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print("Libraries imported.")

Libraries imported.


### 2. Get data from website into a DataFrame

In [30]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Baku_geography_stubs").text
soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-content-ltr")[0].findAll("li"):
    neighborhoodList.append(row.text)
del neighborhoodList[:3]
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df

,Neighborhood
0,Badamdar
1,Bakıxanov
2,Balaxanı
3,Baş Ələt
4,Bibiheybət
5,Biləcəri
6,Bilgəh
7,"Binə, Baku"
8,Binəqədi raion
9,Bukhta-Ilicha


### 3. Get the geographical coordinates

In [31]:
def get_latlng(university):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Baku, Azerbaijan'.format(university))
        lat_lng_coords = g.latlng
    return lat_lng_coords

# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(university) for university in kl_df["Neighborhood"].tolist() ]
coords

[[40.361599648164855, 49.81508025040292],
 [40.43198000000007, 49.95330000000007],
 [40.46255072907915, 49.9343836061088],
 [40.410660000000064, 49.87222000000003],
 [40.33624099687983, 49.82380512464341],
 [40.44440000000003, 49.805660000000046],
 [40.410660000000064, 49.87222000000003],
 [40.410660000000064, 49.87222000000003],
 [40.466731277528304, 49.843503490449706],
 [40.35000000000008, 49.833330000000046],
 [40.372021668998926, 49.84465435423397],
 [40.51789000000008, 50.11390000000006],
 [40.49750000000006, 50.21222000000006],
 [40.283060000000035, 49.28074000000004],
 [40.320350000000076, 50.59199000000007],
 [40.320350000000076, 50.59199000000007],
 [40.316670000000045, 50.583330000000046],
 [40.44053000000008, 50.27612000000005],
 [40.433520000000044, 50.276380000000074],
 [40.38513000000006, 49.94174000000004],
 [40.410660000000064, 49.87222000000003],
 [40.410660000000064, 49.87222000000003],
 [40.410660000000064, 49.87222000000003],
 [40.401200000000074, 50.33402000000006

In [4]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(81, 3)


,University,Latitude,Longitude
0,Badamdar,40.361600,49.815080
1,Bakıxanov,40.431980,49.953300
2,Balaxanı,40.462551,49.934384
3,Baş Ələt,40.410660,49.872220
4,Bibiheybət,40.336241,49.823805
5,Biləcəri,40.444400,49.805660
6,Bilgəh,40.410660,49.872220
7,"Binə, Baku",40.410660,49.872220
8,Binəqədi raion,40.466731,49.843503
9,Bukhta-Ilicha,40.350000,49.833330


### 4. Create a map of Baku with universities superimposed on top


In [5]:
address = 'Baku, Azerbaijan'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Baku,Azerbaijan is  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Baku,Azerbaijan is  40.3754434, 49.8326748.


In [6]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, university in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['University']):
    label = '{}'.format(university)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

## 5. Use the Foursquare API to explore the neighborhoods

In [26]:
CLIENT_ID = 'xxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxx' # your Foursquare Secret
VERSION = '20191009' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxxxxx
CLIENT_SECRET:xxxx


In [8]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['University']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['University', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df

(2083, 7)


,University,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Badamdar,40.361600,49.815080,Beerbaşa,40.363921,49.818902,Brewery
1,Badamdar,40.361600,49.815080,Nakhchivan Restaurant,40.363588,49.819427,Restaurant
2,Badamdar,40.361600,49.815080,Şəki Restoranı,40.357243,49.811641,Restaurant
3,Badamdar,40.361600,49.815080,Bouquet,40.363285,49.820232,Flower Shop
4,Badamdar,40.361600,49.815080,NAR cafe,40.366750,49.815217,Hookah Bar
5,Badamdar,40.361600,49.815080,Buxara,40.362943,49.816071,Comfort Food Restaurant
6,Badamdar,40.361600,49.815080,Fairmont Baku Hotel,40.360448,49.827053,Hotel
7,Badamdar,40.361600,49.815080,IMAX Park Cinema,40.360294,49.825973,Multiplex
8,Badamdar,40.361600,49.815080,Azza,40.365350,49.819585,Dessert Shop
9,Badamdar,40.361600,49.815080,Şəlalə Parkı,40.363310,49.827972,Park


In [9]:
venues_df.groupby(["University"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
University,,,,,,
Badamdar,100,100,100,100,100,100
Bakıxanov,22,22,22,22,22,22
Balaxanı,6,6,6,6,6,6
Baş Ələt,68,68,68,68,68,68
Bibiheybət,50,50,50,50,50,50
Bilgəh,68,68,68,68,68,68
Biləcəri,10,10,10,10,10,10
"Binə, Baku",68,68,68,68,68,68
Binəqədi raion,11,11,11,11,11,11


In [10]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 179 uniques categories.


In [11]:
"University" in venues_df['VenueCategory'].unique()

False

### 6. Analyze Each Neighborhood


In [12]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['University'] = venues_df['University'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2083, 180)


,University,Airport,Airport Food Court,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Bus Station,Bus Stop,Business Service,Café,Campground,Caucasian Restaurant,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cruise,Cupcake Shop,Department Store,Dessert Shop,Diner,Doner Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Lake,Lighthouse,Lounge,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Mosque,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,National Park,New American Restaurant,Nightclub,Office,Other Great Outdoors,Paintball Field,Park,Pharmacy,Photography Studio,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Public Art,Public Bathroom,Racecourse,Racetrack,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,State / Provincial Park,Steakhouse,Street Art,Supermarket,Tatar Restaurant,Tea Room,Tennis Court,Theater,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Ukrainian Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Women's Store
0,Badamdar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Badamdar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Badamdar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Badamdar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Badamdar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [13]:
kl_grouped = kl_onehot.groupby(["University"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(70, 180)


,University,Airport,Airport Food Court,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Bus Station,Bus Stop,Business Service,Café,Campground,Caucasian Restaurant,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cruise,Cupcake Shop,Department Store,Dessert Shop,Diner,Doner Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Lake,Lighthouse,Lounge,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Mosque,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,National Park,New American Restaurant,Nightclub,Office,Other Great Outdoors,Paintball Field,Park,Pharmacy,Photography Studio,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Public Art,Public Bathroom,Racecourse,Racetrack,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,State / Provincial Park,Steakhouse,Street Art,Supermarket,Tatar Restaurant,Tea Room,Tennis Court,Theater,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Ukrainian Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Women's Store
0,Badamdar,0.000000,0.000000,0.0,0.000000,0.00,0.02,0.000000,0.000000,0.000000,0.0000,0.000000,0.01,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.00,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.000000,0.0,0.000000,0.070000,0.000000,0.030000,0.00,0.00000,0.000000,0.01000,0.030000,0.010000,0.01,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.020000,0.010000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.01,0.000000,0.000000,0.000000,0.000000,0.020000,0.0000,0.000000,0.000000,0.00000,0.030000,0.00,0.010000,0.00,0.0000,0.050000,0.01,0.00,0.000000,0.010000,0.000000,0.0,0.000000,0.00,0.010000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.00,0.030000,0.000000,0.00,0.01,0.000000,0.010000,0.000000,0.010000,0.02,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.110000,0.000000,0.0,0.000000,0.010000,0.000000,0.000000,0.010000,0.010000,0.010000,0.00,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.110000,0.010000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.040000,0.00,0.00000,0.00000,0.010000,0.01,0.000000,0.0,0.000000,0.0,0.050000,0.01,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.020000,0.01,0.000,0.000000,0.01,0.01,0.000000
1,Bakıxanov,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.045455,0.000000,0.000000,0.0000,0.045455,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000

In [14]:
len(kl_grouped[kl_grouped["Restaurant"] > 0])


46

In [15]:
kl_mall = kl_grouped[["University","Restaurant"]]

### 7. Cluster Neighborhoods


In [16]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["University"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 1, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [17]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [18]:
kl_merged.rename(columns={"University": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,Badamdar,0.110000,2
1,Bakıxanov,0.090909,2
2,Balaxanı,0.000000,1
3,Baş Ələt,0.073529,2
4,Bibiheybət,0.160000,2


In [19]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("University"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(70, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Badamdar,0.110000,2,40.361600,49.815080
1,Bakıxanov,0.090909,2,40.431980,49.953300
2,Balaxanı,0.000000,1,40.462551,49.934384
3,Baş Ələt,0.073529,2,40.410660,49.872220
4,Bibiheybət,0.160000,2,40.336241,49.823805


In [20]:
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(70, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
45,"Sabunçu, Baku",0.333333,0,40.518680,49.967200
57,Yeni Ələt,0.500000,0,39.941020,49.389520
40,Qaya,0.256410,0,40.347950,49.810660
44,Sabunçu raion,0.333333,0,40.518680,49.967200
42,Rəsulzadə,0.230769,0,40.440686,49.931675
43,Sabail raion,0.277778,0,40.332550,49.815310
38,Qarakosa,0.000000,1,40.046170,49.421600
36,Pirşağı,0.000000,1,40.563880,49.890270
48,Shuvalan,0.043478,1,40.491150,50.182650
33,Pirallahy raion,0.000000,1,40.456800,50.333920


In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [22]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
45,"Sabunçu, Baku",0.333333,0,40.518680,49.967200
57,Yeni Ələt,0.500000,0,39.941020,49.389520
40,Qaya,0.256410,0,40.347950,49.810660
44,Sabunçu raion,0.333333,0,40.518680,49.967200
42,Rəsulzadə,0.230769,0,40.440686,49.931675
43,Sabail raion,0.277778,0,40.332550,49.815310


In [23]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
38,Qarakosa,0.000000,1,40.046170,49.421600
36,Pirşağı,0.000000,1,40.563880,49.890270
48,Shuvalan,0.043478,1,40.491150,50.182650
33,Pirallahy raion,0.000000,1,40.456800,50.333920
23,Korgöz,0.000000,1,40.304300,49.624560
51,Türkan,0.000000,1,40.363210,50.218190
41,Qoşaqışlaq,0.000000,1,40.424780,50.262820
29,Müşfiqabad,0.000000,1,40.466781,49.624997
28,Lökbatan,0.000000,1,40.317410,49.728780
27,Kələzağ,0.000000,1,40.446390,50.228690


In [25]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
67,"Əhmədli, Baku",0.187500,2,40.385130,49.941740
64,Şağan,0.062500,2,40.507910,50.142030
63,"Üçtəpə, Baku",0.073529,2,40.410660,49.872220
54,Yasamal raion,0.090000,2,40.380260,49.830330
62,Zəfəran,0.105263,2,40.484940,50.156560
61,"Zığ, Baku",0.062500,2,40.367100,49.985650
49,Sulutəpə,0.100000,2,40.435910,49.761750
59,Zağulba Bağları,0.142857,2,40.542770,50.089980
50,Suraxanı raion,0.066667,2,40.416511,49.993335
58,Zabrat,0.142857,2,40.479760,49.950540


## Result

In the central district of Baku, the highest number of restaurants in cluster 2 and the average number in cluster 0 are seen. Since there is little or no competition in existing restaurants, this restaurant represents a great opportunity to open re-launchers and high potential areas. . Meanwhile, restaurants in cluster 2 suffer from intense competition due to the high restaurant density. On the other hand, this shows that most of the extreme restaurants are located in the city center and there are still very few restaurants in the suburban area. Therefore, this project proposes that property developers benefit from these findings in which there is little or no competition to open new restaurants in the neighborhoods in cluster 1. competition in neighborhoods in cluster 0. Finally, property developers are advised to avoid restaurants in cluster 2;